# Reqcuired Libraries

In [ ]:
import urllib
import urllib.request
import bs4 as bs
import pandas as pd
import csv
import os
import re

# Asign variables to urls

In [ ]:
url1 = The review Page of attraction #1
url2 = The review Page of attraction #2

# Important!
It is essential to use the Review page as the URL and not the attraction page. Otherwise, reviews only display a portion of the text.

# Complete Code

In [ ]:
# List of URls
WebSites = [url1, url2, url3, etc...]
exten = '.csv'
File = r'Your File Here'
#Progressor
q = 1
#Loop through each URL
for theurl in WebSites:
    #Create empty lists that will feed the pandas data frame
    location = []
    date = []
    rating = []
    texto = []
    atractions = []
    contribution = []
    # Title of output csv
    t = theurl.split('-')[4:]
    tit = ''.join(t)
    title = tit.strip('.html#REVIEWS')
    df = pd.DataFrame()
    # Get Soup
    thepage = urllib.request.urlopen(theurl)
    sopa = bs.BeautifulSoup(thepage, "html.parser")
    while True:
        my_div = sopa.find_all("div", class_="reviewSelector")
        # Loop through each review window
        for div in my_div:
            user = div.find_all("div", class_ = "username")
            for use in user:
                nuser = str(use.text)
                # If user dosent have personal information, skip (It is troublesome to deal with these users)
                if nuser.strip() != "A TripAdvisor Member":
                    # Acquire Country of origin of user
                    loc = div.find_all("div", class_="location")
                    for l in loc:
                        location.append(l.string)
                    span = div.find_all("div", class_='rating reviewItemInline')
                    # Acquire Date of review
                    for s in span:
                        if div.find(attrs={'class': 'ratingDate relativeDate'}) in s:
                            pre = div.find_all(attrs={'class':"ratingDate relativeDate"})  
                            for pr in pre:
                                date.append(pr.attrs['title'])            
                        else:  
                            al = div.find_all(attrs={'class':'ratingDate'})
                            for ka in al:
                                date.append(ka.string)   
                    # Acquire text of Review            
                    tex = div.find_all("div", class_= "entry")
                    for t in tex:
                        texto.append(t.text)    
                    toFind = re.compile("(bubble_[0-9])+")
                    for s in div.findAll("span", {"class":toFind}):
                        ran = s
                        rating.append(ran)
                    # Acquire number of reviews posted by user and how helpfull they are   
                    badge = div.find_all("div", class_= "memberBadging g10n")
                    for b in badge:
                        if div.find("div", class_= "helpfulVotesBadge badge no_cpu") in b:
                            helpf = div.find_all("div", class_= "helpfulVotesBadge badge no_cpu")
                            for h in helpf:
                                contribution.append(h.text)
                        else:
                            contribution.append("0 helpful votes")    
                    att = div.find_all('div', class_= "reviewerBadge badge")
                    for a in att:
                        atractions.append(a.text)
                else: pass         
                
        link = sopa.find_all(attrs={"class": "nav next rndBtn ui_button primary taLnk"})
        if len(link) == 0:
            # Clean up data to make it more usable
            country = []
            for item in location:
                country.append(item.split(",")[-1])
            pais = []      
            for it in country:
                pais.append(it.strip("\n"))    
            text = []
            for rev in texto:
                t = rev.strip("\n")
                ts = str(t)
                text.append(ts)
                helpful_votes = []
            for votes in contribution:
                streper = votes.strip("\n")
                helpful_votes.append(streper.split()[0])
                reviewed_at = []
            for rev in atractions:
                reveti = rev.strip("\n")
                reviewed_at.append(reveti.split()[0])
            # Convert image name of rating into a value from 1 to 5    
            Stars = []
            for item in rating:
                x = str(item)
                if x.endswith('bubble_1"></span>') == True:
                    Stars.append(1)
                elif x.endswith('bubble_2"></span>') == True:
                    Stars.append(2)
                elif x.endswith('bubble_3"></span>') == True:
                    Stars.append(3)
                elif x.endswith('bubble_4"></span>') == True:
                    Stars.append(4)
                else:
                    Stars.append(5)  
            # Insert Data into Pandas Data Frame        
            df["Date"] = date
            df["Country of Origin"] = pais
            df["Review"] = text
            df["Rating"] = Stars
            df["reviewed attractions"] = reviewed_at
            df["helpful votes"] = helpful_votes 
            new = []
            for item in df['Date']:
                x = item.strip('Reviewed')
                y = x.strip('\n')
                new.append(y)
            df.drop('Date', axis = 1)
            # Convert Date into Month and Year
            df['Date'] = new
            df['Month'] = df['Date'].apply(lambda x: x.split()[0])
            df['Year'] = df['Date'].apply(lambda x: x.split()[-1])
            # Create path of csv file
            path = os.path.join(File, title)
            df.to_csv(path + exten)
            # Progressor label
            print("Page {} out of {}".format(q, len(WebSites)))
            q = q+1
            break
        # Feed next Review Page    
        else:
            sopa = bs.BeautifulSoup(urllib.request.urlopen("http://www.tripadvisor.com" + link[0].get('href')),"html.parser")
